In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
raw_data = np.loadtxt(open("EEG.csv", "rb"), delimiter=",", skiprows=0)
X_raw = raw_data[:,:-1]

In [3]:
# Data cleaning on X
upper_threshold = 4800
lower_threshold = 3800
too_big = X_raw>upper_threshold
too_small = X_raw<lower_threshold
X_cleaned = np.copy(X_raw)
X_cleaned[too_big] = upper_threshold
X_cleaned[too_small] = lower_threshold

In [4]:
X = (X_cleaned - np.mean(X_cleaned)) / np.std(X_cleaned)
y = raw_data[:,-1]

In [5]:
X.shape

(14980, 14)

In [56]:
num_train = int(X.shape[0] * 0.8)
num_val = int(X.shape[0] * 0.1)
X_train = X[:num_train,:]
y_train = y[:num_train]
X_val = X[num_train:num_train+num_val,:]
y_val = y[num_train:num_train+num_val]
X_test = X[num_train+num_val:,:]
y_test = y[num_train+num_val:]

In [107]:
model = Sequential()
model.add(Dense(256, activation = 'relu', input_dim=X.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
adam = optimizers.Adam(lr=0.05,decay=0)
model.compile(optimizer = adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [108]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
11984/11984 [==============================] - 1s - loss: 0.7157 - acc: 0.5337       

In [109]:
score = model.evaluate(X_val, y_val,verbose=1, batch_size=64)
print(score)

  64/1498 [>.............................] - ETA: 6s[0.65069388690714203, 0.60747663575275557]


In [110]:
score = model.evaluate(X_test, y_test,verbose=1, batch_size=64)
print(score)

1344/1498 [=========================>....] - ETA: 0s[0.68056822380172877, 0.46395193615328961]


In [111]:
np.mean(model.predict(X_val)>0.5)

0.4145527369826435

In [112]:
np.mean(model.predict(X_test)>0.5)

0.56208277703604803

In [114]:
np.mean(y_val)

0.12483311081441922